# An Introduction to Classification

* Contingency Tables Revisited
    * odds
    * other hypothesis tests
* Logistic Regression
    * log odds
    * connection to linear regression
    * interpretation
    * parameters in scikit-learn
    * decision function
    * precision 
    * recall
    * f1 score
    * MCC
    * 